# SedonaDB + lonboard visualizations

In [1]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [2]:
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=divisions/type=division_area/"
)

In [3]:
df.show(10)

┌─────────────────┬────────────────┬────────────────┬───┬────────────────┬────────┬────────────────┐
│        id       ┆    geometry    ┆      bbox      ┆ … ┆ is_territorial ┆ region ┆   division_id  │
│       utf8      ┆    geometry    ┆     struct     ┆   ┆     boolean    ┆  utf8  ┆      utf8      │
╞═════════════════╪════════════════╪════════════════╪═══╪════════════════╪════════╪════════════════╡
│ aeaf43ac-7b6f-… ┆ POLYGON((-46.… ┆ {xmin: -46.55… ┆ … ┆ true           ┆ BR-SP  ┆ 4f608d01-de34… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 13ee265d-ad42-… ┆ POLYGON((-46.… ┆ {xmin: -46.56… ┆ … ┆ true           ┆ BR-SP  ┆ f87b7de2-795f… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 74d4ed65-59d2-… ┆ POLYGON((-46.… ┆ {xmin: -46.55… ┆ … ┆ true           ┆ BR-SP  ┆ 9cdde1c7-98be… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌

In [4]:
df.to_view("division_area")

In [5]:
sd.sql("""
SELECT
    COUNT(*)
FROM division_area
""").show()

┌──────────┐
│ count(*) │
│   int64  │
╞══════════╡
│  1035749 │
└──────────┘


In [6]:
nova_scotia_bbox_wkt = (
    "POLYGON((-66.5 43.4, -66.5 47.1, -59.8 47.1, -59.8 43.4, -66.5 43.4))"
)
ns = sd.sql(f"""
SELECT
    country, region, names, geometry
FROM division_area
WHERE
    ST_Intersects(
        geometry,
        ST_SetSRID(ST_GeomFromText('{nova_scotia_bbox_wkt}'), 4326)
    )
""").to_memtable()

In [7]:
ns.to_view("ns_divisions")

In [8]:
df = sd.sql("""
SELECT UNNEST(names), geometry
FROM ns_divisions
WHERE region = 'CA-NS'
""")

In [9]:
import lonboard

lonboard.viz(df)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [10]:
# FWIW you could make the visualization a lot prettier by using a PolygonLayer 
# and using some sort of data-driven styling. Maybe render color based on height?

In [11]:
from lonboard import Map, PolygonLayer

layer = PolygonLayer(
    df,
    get_fill_color=[255, 0, 0],
    get_line_color=[0, 100, 100, 150],
)
m = Map(layer)

In [12]:
m

Map(custom_attribution='', layers=(PolygonLayer(get_fill_color=[255, 0, 0], get_line_color=[0, 100, 100, 150],…